# RiskLens VCDB Data Set for Hiring Exercise

This notebook contains the process for building the data set for the RiskLens Data Science hiring exercise.

It uses VCDB data cloned from the VCDB repository (https://github.com/vz-risk/VCDB) on March 26.  The most recent commit is:

``` bash
VCDB $ git log | head

commit 246e453741db4c2c73b71654c4afea54eb4dfc3a
Author: Gabriel Bassett <gabe@infosecanalytics.com>
Date:   Sun Mar 3 09:49:12 2019 -0600
```

It also makes use of the verispy package (https://github.com/RiskLens/verispy), developed by RiskLens. We use verispy version 0.1.2, available via pypi: https://pypi.org/project/verispy/ 

To limit the time that we wish applicants to spend on this exercise, we have simplified the data set significantly, and chosen to focus on medical-related incident data.

In [1]:
import pandas as pd
from verispy import VERIS
pd.options.mode.chained_assignment = None

In [5]:
# instantiate object, convert json data to dataframe
data_dir = '../../VCDB/data/json/validated'
v = VERIS(json_dir=data_dir)
vdf = v.json_to_df()

In [6]:
vdf.shape

(8198, 2330)

In [7]:
vdf.head()

,action.Environmental,action.Error,action.Hacking,action.Malware,action.Misuse,action.Physical,action.Social,action.Unknown,action.environmental.notes,action.environmental.variety.Deterioration,...,victim.revenue.iso_currency_code.XPF,victim.revenue.iso_currency_code.YER,victim.revenue.iso_currency_code.ZAR,victim.revenue.iso_currency_code.ZMK,victim.revenue.iso_currency_code.ZWD,victim.secondary.amount,victim.secondary.notes,victim.secondary.victim_id,victim.state,victim.victim_id
0,False,False,False,False,True,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,NJ,"C.R. Bard, Inc."
1,False,True,False,False,False,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,British Columbia Ministry of Finance
2,False,False,True,True,False,False,True,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
3,False,False,True,False,False,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,FL,Camberwell High School
4,False,False,True,False,False,False,False,False,NaN,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,Loyalist Certification Services Exams


Filter for medical industry:

In [8]:
med = vdf[vdf['victim.industry2.62']]

In [9]:
med.shape

(2252, 2330)

## Build Data Set

Have one script to build the data. This isn't terribly pretty, but it works for a one-off situation

In [10]:
def collapse_top_level_enum(df, enums, is_asset=False, is_asset_sub=False):
    
    new_df = pd.DataFrame()
    for enum in enums:
        if is_asset:
            sfx = enum.split('.')[2].lower()
        elif is_asset_sub:
            sfx = enum.split('.')[3].lower()
        else: 
            sfx = enum.split('.')[1].lower()
        new_df[enum] = df[enum].apply(lambda x: sfx if x else '')
        
    
    new_df['outcol'] = None
    for index, row in new_df.iterrows():
        for enum in enums:
            val = row[enum]
            if len(val) > 0:
                new_df.loc[index, 'outcol'] = val
                # only do first one
                break
                
    return new_df

In [11]:
# filter for medical data
med = vdf[vdf['victim.industry2.62']]

# identifying info
use_data = med[['incident_id'] + [col for col in med.columns if col.startswith('timeline.incident')]]

# get the actors (first one only)
actor_enums = [col for col in med.columns if col.startswith('actor') and len(col.split('.')) == 2]
actors = collapse_top_level_enum(med, actor_enums)
use_data['actor'] = actors['outcol']
actors_subs = {}
for enum in actor_enums:
    full_enum = '.'.join((enum.lower(), 'variety'))
    actor_subenums = [col for col in med.columns if col.startswith(full_enum) and len(col.split('.')) == 4]
    actors_subs[enum.lower().split('.')[1]] = actor_subenums
        

# get the action (first one only)
action_enums = [col for col in med.columns if col.startswith('action') and len(col.split('.')) == 2]
actions = collapse_top_level_enum(med, action_enums)
use_data['action'] = actions['outcol']

# get the attribute (first one only)
#attribute_enums = [col for col in med.columns if col.startswith('attribute') and len(col.split('.')) == 2]
#attributes = collapse_top_level_enum(med, attribute_enums)
#use_data['attribute'] = attributes['outcol']

use_data['attribute.confidentiality'] = med['attribute.Confidentiality']
use_data['attribute.integrity'] = med['attribute.Integrity']
use_data['attribute.availability'] = med['attribute.Availability']

# get the asset class
asset_enums = [col for col in med.columns if col.startswith('asset.variety') and len(col.split('.')) == 3]
assets = collapse_top_level_enum(med, asset_enums, is_asset=True)
use_data['asset'] = assets['outcol']

asset_subenums = [col for col in med.columns if col.startswith('asset.assets.variety') and len(col.split('.')) == 4]
assets_subs = collapse_top_level_enum(med, asset_subenums, is_asset_sub=True)
use_data['asset.variety'] = assets_subs['outcol']


use_data['confidentiality.medical_records'] = med['attribute.confidentiality.data.amount.Medical']
use_data['confidentiality.payment_records'] = med['attribute.confidentiality.data.amount.Payment']
use_data['confidentiality.personal_records'] = med['attribute.confidentiality.data.amount.Personal']
use_data['confidentiality.total_record_count'] = med['attribute.confidentiality.data_total']
fill_conf_na = ['confidentiality.medical_records','confidentiality.payment_records', 'confidentiality.personal_records', 'confidentiality.total_record_count']
use_data[fill_conf_na] = use_data[fill_conf_na].fillna(value=0)


# get employee count
emplct_enums = [col for col in med.columns if col.startswith('victim.employee_count') and len(col.split('.')) == 3]
emplcts = collapse_top_level_enum(med, emplct_enums, is_asset=True)
use_data['victim.employee_count'] = emplcts['outcol']

# get victim location
use_data['victim.state'] = med['victim.state']
country_enums = [col for col in med.columns if col.startswith('victim.country') and len(col.split('.')) == 3]
countries = collapse_top_level_enum(med, country_enums, is_asset=True)
use_data['victim.country'] = countries['outcol'].apply(lambda x: x.upper())


use_data['victim.victim_id'] = med['victim.victim_id']
use_data['summary']  = med['summary']
use_data['reference'] = med['reference']

In [12]:
use_data.head(20)

,incident_id,timeline.incident.day,timeline.incident.month,timeline.incident.time,timeline.incident.year,actor,action,attribute.confidentiality,attribute.integrity,attribute.availability,...,confidentiality.medical_records,confidentiality.payment_records,confidentiality.personal_records,confidentiality.total_record_count,victim.employee_count,victim.state,victim.country,victim.victim_id,summary,reference
5,F8E9958F-1964-43C2-9FFD-18626E762388,20.0,1.0,NaN,2016,external,malware,True,True,False,...,0.0,0.0,0.0,0.0,unknown,NaN,AU,Royal Melbourne Hospital,"A report this week in The Age said a ""computer...",http://www.databreaches.net/au-hack-attack-on-...
11,CC5E5B26-62A1-42C5-9C12-FC25F4267271,11.0,6.0,NaN,2010,external,physical,True,False,True,...,1105.0,0.0,0.0,1105.0,11 to 100,KS,US,Occupational Health Partners,NaN,NaN
19,D305CA4D-0909-4F8B-9BA5-4CC3BEA5D33A,NaN,10.0,NaN,2014,internal,error,True,False,False,...,1778.0,0.0,0.0,1778.0,101 to 1000,MN,US,Northfield Hospital & Clinics,Northfield Hospital & Clinics is in the proces...,http://www.southernminn.com/northfield_news/ar...
25,782906c0-ee92-11e7-b683-c1703914e4a4,NaN,NaN,NaN,2014,partner,error,True,False,False,...,500.0,0.0,0.0,500.0,unknown,NaN,AU,NaN,The website was automatically storing naked ph...,https://au.news.yahoo.com/nsw/a/35742204/bondi...
29,BDD1DE7E-4DEB-428D-A97E-C7D650238C5D,3.0,2.0,NaN,2012,external,physical,True,False,True,...,7757.0,0.0,0.0,7757.0,unknown,KS,US,Kansas Department on Aging,NaN,NaN
32,EE0E8BA2-7D15-4C62-AC49-6405DA49EA81,5.0,3.0,NaN,2013,external,physical,True,False,True,...,0.0,0.0,0.0,539.0,unknown,TN,US,Integrity Oncology,Integrity Oncology's business associate North ...,http://www.hhs.gov/ocr/privacy/hipaa/administr...
33,B124687D-155B-4EED-BACF-E90D902E8F93,27.0,8.0,NaN,2015,internal,misuse,True,False,False,...,164.0,0.0,0.0,164.0,large,CA,US,Valley Children's Hospital,Doctors accessed medical records in attempts t...,http://www.fresnobee.com/news/local/article380...
34,761a7e70-98e5-11e7-a6db-535ccd646699,5.0,7.0,NaN,2017,external,malware,True,True,True,...,0.0,0.0,0.0,0.0,101 to 1000,NaN,US,Caro Community Hospital,Ransomware attack on hospital,http://www.wnem.com/story/35903543/caro-hospit...
42,E61A58E8-D8F3-453C-B897-30A9D7C8A4D4,3.0,6.0,NaN,2013,external,physical,True,False,True,...,1167.0,0.0,0.0,1167.0,101 to 1000,OH,US,"Community Support Services, Inc.",Theft of email results in exposure of 1167 rec...,https://ocrportal.hhs.gov/ocr/breach/breach_re...
45,03B94103-B239-4093-80CB-AC9430E6CE8F,4.0,5.0,NaN,2010,external,physical,True,False,True,...,3621.0,0.0,0.0,3621.0,1 to 10,MA,US,Massachusetts Eye and Ear Infirmary,NaN,NaN


In [13]:
use_data.shape

(2252, 22)

Save to CSV:

In [14]:
use_data.to_csv('vcdb_medical_simplified.csv', index=False)

In [15]:
use_data.columns

Index(['incident_id', 'timeline.incident.day', 'timeline.incident.month',
       'timeline.incident.time', 'timeline.incident.year', 'actor', 'action',
       'attribute.confidentiality', 'attribute.integrity',
       'attribute.availability', 'asset', 'asset.variety',
       'confidentiality.medical_records', 'confidentiality.payment_records',
       'confidentiality.personal_records',
       'confidentiality.total_record_count', 'victim.employee_count',
       'victim.state', 'victim.country', 'victim.victim_id', 'summary',
       'reference'],
      dtype='object')